In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from openpyxl import load_workbook

In [2]:
nosdra_df = pd.read_excel("./data/Nosdra (Scraped, Not Clean).xlsx", encoding="ISO-8859-1")

In [3]:
nosdra_df[["shell_jiv_number", "eni_jiv_number"]].head()

,shell_jiv_number,eni_jiv_number
0,NaN,SAR-360-486-104
1,NaN,SAR-488-668-546
2,1571105.0,NaN
3,1350803.0,NaN
4,809483.0,NaN


In [4]:
nosdra_df.shell_jiv_number.notnull().sum()

836

In [5]:
nosdra_df.eni_jiv_number.notnull().sum()

721

In [6]:
nosdra_df.shape[0]

13017

### Verify nosdra company does not have any shell and eni jiv ids

In [7]:
nosdra_df[(nosdra_df.nosdra_company.isin(["NAOC", "SPDC"])) & (nosdra_df.shell_jiv_number.notnull()) & (nosdra_df.eni_jiv_number.notnull())]

,id,nosdra_id,nosdra_status,nosdra_unverified_by_nosdra,nosdra_zonal_offce,nosdra_company,nosdra_incident_number,nosdra_incident_date,nosdra_report_date,nosdra_containment,...,shell_estimated_spill_volume,shell_clean_up_status,shell_comments,eni_jiv_number,eni_facility_equipment,eni_area,eni_lga,eni_date_of_incident_spill,eni_incident_spill_cause,eni_estimated_spill_volume


In [8]:
nosdra_df = nosdra_df[nosdra_df.nosdra_company.isin(["NAOC", "SPDC"])]

In [9]:
nosdra_df.shape[0]

8125

In [10]:
nosdra_df.to_excel("./data/NOSDRA_Shell_Eni_only.xlsx")

In [11]:
nosdra_df.shell_jiv_number.isnull().sum()

8125

In [12]:
nosdra_df.eni_jiv_number.isnull().sum()

8125

In [13]:
nosdra_df.shape[0]

8125

In [14]:
nosdra_df[nosdra_df.nosdra_estimated_spill_area > 1].shape

(32, 55)

In [15]:
nosdra_df.nosdra_jiv_date.isnull().sum()

1787

In [16]:
nosdra_df.nosdra_present_at_jiv.isnull().sum()

8125

### remove unnecessary columns

In [17]:
nosdra_df.loc[:, :"nosdra_last_updated_by"].isnull().sum()

id                                         0
nosdra_id                                  0
nosdra_status                              0
nosdra_unverified_by_nosdra                0
nosdra_zonal_offce                      8125
nosdra_company                             0
nosdra_incident_number                  2259
nosdra_incident_date                     442
nosdra_report_date                      5874
nosdra_containment                      8125
nosdra_estimated_quantity                  0
nosdra_quantity_recovered                  0
nosdra_spill_stop_date                  6701
nosdra_type_of_facility                 5503
nosdra_cause                            1087
nosdra_initial_containment_mesures      8125
nosdra_site_location_name                 11
nosdra_latitude                            0
nosdra_logitude                            0
nosdra_lga                              2377
nosdra_estimated_spill_area                0
nosdra_spill_area_habitat               1072
nosdra_des

In [18]:
nosdra_df.loc[:,"shell_jiv_number":"eni_estimated_spill_volume"].isnull().sum()

shell_jiv_number                8125
shell_date_reported             8125
shell_incident_site             8125
shell_jiv_date                  8125
shell_terrain                   8125
shell_cause                     8125
shell_estimated_spill_volume    8125
shell_clean_up_status           8125
shell_comments                  8125
eni_jiv_number                  8125
eni_facility_equipment          8125
eni_area                        8125
eni_lga                         8125
eni_date_of_incident_spill      8125
eni_incident_spill_cause        8125
eni_estimated_spill_volume      8125
dtype: int64

### remove empty columns

In [19]:
nosdra_df = nosdra_df.loc[:, :"nosdra_last_updated_by"]

In [20]:
del nosdra_df["nosdra_zonal_offce"]
del nosdra_df["nosdra_containment"]
del nosdra_df["nosdra_initial_containment_mesures"]
del nosdra_df["nosdra_present_at_jiv"]
del nosdra_df["nosdra_post_impact_assesment_date"]
del nosdra_df["nosdra_remidiation_start"]
del nosdra_df["nosdra_remidiation_end"]

In [21]:
nosdra_df.head()

,id,nosdra_id,nosdra_status,nosdra_unverified_by_nosdra,nosdra_company,nosdra_incident_number,nosdra_incident_date,nosdra_report_date,nosdra_estimated_quantity,nosdra_quantity_recovered,...,nosdra_form_b_date,nosdra_form_c_date,nosdra_jiv_date,nosdra_clean_up_date,nosdra_clean_up_methods,nosdra_post_clean_up_inspection_date,nosdra_final_sampling_date,nosdra_certificate_date,nosdra_certificate_number,nosdra_last_updated_by
3260,3261,1682,confirmed,0,NAOC,2001/LAR/017,2001-05-20 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-06-29 00:00:00,2013-02-11 00:00:00,NOSDRA/RS/000908,"2016-01-08 AB011, 2015-04-14 AB006, NOSDRA"
3261,3262,1683,confirmed,0,NAOC,2001/LAR/049,2001-10-03 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-06-29 00:00:00,NaN,NaN,NOSDRA
3262,3263,1684,confirmed,0,NAOC,NaN,2002-06-18 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2009-05-07 00:00:00,NaN,NaN,NOSDRA
3263,3264,1685,confirmed,0,NAOC,NaN,2002-07-24 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2009-05-06 00:00:00,NaN,NaN,NOSDRA
3264,3265,1686,confirmed,0,NAOC,NaN,2002-11-17 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2009-05-06 00:00:00,NaN,NaN,NOSDRA


In [22]:
nosdra_df.to_excel("./data/NOSDRA_Shell_Eni_only.xlsx")

In [23]:
nosdra_df.nosdra_incident_number.isnull().sum()

2259

In [24]:
nosdra_df[["nosdra_incident_number", "nosdra_company"]].groupby(["nosdra_company"]).count()

,nosdra_incident_number
nosdra_company,
NAOC,3747
SPDC,2119


In [25]:
nosdra_df[nosdra_df.nosdra_company == "SPDC"].nosdra_incident_number

7088       1990_00144
7089       1990_00145
7090       1991_00137
7091       1994_00003
7092       1994_00129
7093       2001_00108
7094       2008_00234
7095       2003_00197
7096       2004_00155
7097       2004_00168
7098       2004_00226
7099       2006_00001
7100       2006_00005
7101       2006_00004
7102       2006_00038
7103       2006_00006
7104       2006_00016
7105       2006_00011
7106       2006_00013
7107       2006_00026
7108       2006_00025
7109       2006_00031
7110       2006_00021
7111       2006_00037
7112       2006_00048
7113       2006_00044
7114       2006_00043
7115       2006_00047
7116       2006_00061
7117       2006_00050
             ...     
12893    2014_1285421
12894    2016_1628051
12895    2016_1668115
12896    2015_1357611
12899    2015_1440709
12900    2014_1194724
12907    2016_1658680
12908    2016_1677744
12910    2015_1514020
12911    2014-1309037
12912         1399539
12913         1590758
12914    2015_1514020
12915         1388396
12916     

In [26]:
nosdra_df[nosdra_df.nosdra_company == "NAOC"].nosdra_incident_number.head()

3260    2001/LAR/017
3261    2001/LAR/049
3262             NaN
3263             NaN
3264             NaN
Name: nosdra_incident_number, dtype: object

### investigate only Shell Nosdra data

In [27]:
id_number_regex = re.compile(r"((\d+)[_/-])?[0]{0,5}(\d+)")

years = [np.nan] * nosdra_df.shape[0]
shell_jivs = [np.nan] * nosdra_df.shape[0]

for i in range(nosdra_df.shape[0]):
    company = nosdra_df.iloc[i].nosdra_company
    incident_date = nosdra_df.iloc[i].nosdra_incident_date
    number = nosdra_df.iloc[i].nosdra_incident_number
    if type(number) != float:
        if company == "SPDC":
            match = re.match(id_number_regex, str(number))
            if match:
                year = match.group(2)
                number = match.group(3)
                shell_jivs[i] = number
                if str(year) == "None":
                    if type(incident_date) != float:
                        years[i] = incident_date.year
                else:
                    if year[:1] == "3":
                        year = "2" + year[1:]
                    years[i] = year
                
nosdra_df["shell_jiv_extracted"] = pd.Series(shell_jivs).values
nosdra_df["year_of_spill"] = pd.Series(years).values
nosdra_df["year_of_spill"] = pd.to_datetime(nosdra_df["year_of_spill"], format="%Y")

In [28]:
shell_df = nosdra_df[nosdra_df.nosdra_company == "SPDC"]
shell_df.shape, shell_df.year_of_spill.notnull().sum()

((2922, 34), 2108)

### take only oil spills after 2006

In [29]:
from datetime import date
shell_df = shell_df[shell_df.year_of_spill >= date(2006, 1, 1)]

In [30]:
shell_df.head()

,id,nosdra_id,nosdra_status,nosdra_unverified_by_nosdra,nosdra_company,nosdra_incident_number,nosdra_incident_date,nosdra_report_date,nosdra_estimated_quantity,nosdra_quantity_recovered,...,nosdra_jiv_date,nosdra_clean_up_date,nosdra_clean_up_methods,nosdra_post_clean_up_inspection_date,nosdra_final_sampling_date,nosdra_certificate_date,nosdra_certificate_number,nosdra_last_updated_by,shell_jiv_extracted,year_of_spill
7094,7095,5690,confirmed,0,SPDC,2008_00234,2001-10-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,2012-07-09 00:00:00,NaN,NaN,NOSDRA,234,2008-01-01
7099,7100,5695,confirmed,0,SPDC,2006_00001,2005-01-02 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,1,2006-01-01
7100,7101,5696,confirmed,0,SPDC,2006_00005,2005-01-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,5,2006-01-01
7101,7102,5697,confirmed,0,SPDC,2006_00004,2005-01-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,4,2006-01-01
7102,7103,5698,confirmed,0,SPDC,2006_00038,2005-01-05 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,2009-12-08 00:00:00,NOSDRA/RS/000349,"2016-01-05 AB008, NOSDRA",38,2006-01-01


### compare > 2006 Nosdra Shell spills to what the company reported

In [31]:
df = pd.read_csv("./data/EniShell_transformed.csv", sep="¬")
df_shell = df[df.company == "SPDC"]
df_shell.shape[0]

/home/katia/.virtualenvs/securaxis/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


865

In [32]:
shell_jivs = set(df_shell.jiv_number)
nosdra_shell_jivs = set(shell_df.shell_jiv_extracted)

#### How many oil spills were reported by both?

In [33]:
repeated_jivs = shell_jivs.intersection(nosdra_shell_jivs)
len(repeated_jivs)

662

In [34]:
shell_df[~shell_df.shell_jiv_extracted.isin(repeated_jivs)].head()

,id,nosdra_id,nosdra_status,nosdra_unverified_by_nosdra,nosdra_company,nosdra_incident_number,nosdra_incident_date,nosdra_report_date,nosdra_estimated_quantity,nosdra_quantity_recovered,...,nosdra_jiv_date,nosdra_clean_up_date,nosdra_clean_up_methods,nosdra_post_clean_up_inspection_date,nosdra_final_sampling_date,nosdra_certificate_date,nosdra_certificate_number,nosdra_last_updated_by,shell_jiv_extracted,year_of_spill
7094,7095,5690,confirmed,0,SPDC,2008_00234,2001-10-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,2012-07-09 00:00:00,NaN,NaN,NOSDRA,234,2008-01-01
7099,7100,5695,confirmed,0,SPDC,2006_00001,2005-01-02 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,1,2006-01-01
7100,7101,5696,confirmed,0,SPDC,2006_00005,2005-01-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,5,2006-01-01
7101,7102,5697,confirmed,0,SPDC,2006_00004,2005-01-04 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOSDRA,4,2006-01-01
7102,7103,5698,confirmed,0,SPDC,2006_00038,2005-01-05 00:00:00,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,2009-12-08 00:00:00,NOSDRA/RS/000349,"2016-01-05 AB008, NOSDRA",38,2006-01-01


In [35]:
stats_file = "./data/Statistics.xlsx"

def update_excel(new_df, sht_name):
    book = load_workbook(stats_file)

    writer = pd.ExcelWriter(stats_file, engine = 'openpyxl')
    writer.book = book

    new_df.to_excel(writer, sheet_name = sht_name)
    writer.save()
    writer.close()

In [36]:
update_excel(shell_df[~shell_df.shell_jiv_extracted.isin(repeated_jivs)], "Nosdra Shell Spills NOT REPORTED By SHELL")

/home/katia/.virtualenvs/securaxis/lib/python3.5/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
